# 0) Import and install libraries

In [ ]:
use_google_colab = True
if use_google_colab:
    from google.colab import drive
    drive.mount('/gdrive')
    %cd "/gdrive/MyDrive/Colab Notebooks/Scientific Computing Tools for Advanced Mathematical Modeling/FinalProject"
    !pip install tqdm
    !pip install FDApy
    import FDApy
    from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
    from FDApy.preprocessing.dim_reduction.fpca import UFPCA
    from FDApy.visualization.plot import plot

import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import os
import matplotlib.pyplot as plt
import random
from scipy.stats import multivariate_normal
from sklearn.cluster import KMeans
from time import time
from sklearn.metrics.pairwise import euclidean_distances
from sys import getsizeof

from logger import logger, TeseoLogger

# Random seed for reproducibility
seed = 427

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)

# 1) Load the dataset:

The dataset is composed of:
*   coordinates of points
*   UAC 
*   IIR
*   aligned signals
*   signals



In [ ]:
df_og = pd.read_csv('LAsignals.csv')
# remove index column
df_og = df_og.iloc[:,-len(df_og.columns.values)+1:]
# tidy the dataset
start_index = 6 # index used to select only the signals from df
signals = df_og.iloc[:,start_index:].values
coordinates = df_og.iloc[:,:3]
coordinates = coordinates.values.astype('<f4')
UAC = df_og.iloc[:,3:5].values
IIR = df_og.iloc[:,5].values.astype('<f2')
aligned_signals = pd.read_csv('aligned_LAsignals.csv', header = None)


# 2) Voronoi Tessellation algorithm

**Hyperparameters:**

* `B`: number of bootstrap iterations
* `n`: number of nuclei
* `p`: used for dimensionality reduction (e.g. if p=3 we end up with vectors with 3 components)
* `K`: number of classes (always equal to 2)

In [ ]:
def teseo(n, threshold, B = 500, K = 2):
  log = TeseoLogger(os.path.join(os.getcwd(), "teseo_logs"), "log.txt", encoding = ["threshold", "iteration_time", "dimensionality_reduction", "average_normalized_entropy", "distance_matrix_size", "number_of_nucei", "num_of_iterations"])
  # random.seed(0)
  t0 = time()
  classification_matrix = np.zeros((coordinates.shape[0], 1)).astype('<i2')
  average_normalized_entropy = np.zeros((B,1))
  for b in tqdm(range(B)):
    t1 = time()
    nuclei_indices = np.array(random.sample(list(np.arange(coordinates.shape[0])), n)) 
    nuclei_coordinates = np.zeros((n, coordinates.shape[1])) 
    nuclei_coordinates = coordinates[nuclei_indices, :]
    distances = euclidean_distances(coordinates, nuclei_coordinates)
    min_index = np.argmin(distances, axis = 1)
    log.write("distance_matrix_size" + " " + str(getsizeof(distances)/1000000000))
    weights = np.zeros(coordinates.shape[0])
    idx = np.arange(coordinates.shape[0])
    weights[nuclei_indices] = 1
    del distances
    representative = np.zeros((n, aligned_signals.shape[1]))  
    representative = aligned_signals.values[nuclei_indices]
    del weights
    fd = np.diff(representative)
    teta = 5
    d = np.sqrt(euclidean_distances(representative, representative)**2 + teta*euclidean_distances(fd, fd)**2)
    c = np.sum(d, axis = 1)
    log.write("threshold" + " " + str(threshold))
    tau = np.quantile(c, threshold)
    labels = np.zeros_like(c, dtype = int)
    labels[c < tau] = 1
    labels = labels.reshape(-1,)
    idx = np.arange(coordinates.shape[0])
    for i in range(n):
      if labels[i] == 1:
        classification_matrix[idx[min_index == i]] += 1
    p1 = classification_matrix / (b+1)
    p0 = 1 - p1
    v0 = np.zeros((coordinates.shape[0], 1))
    v1 = np.zeros((coordinates.shape[0], 1))
    v0[p0 != 0] = p0[p0 != 0] * np.log(p0[p0 != 0])
    v1[p1 != 0] = p1[p1 != 0] * np.log(p1[p1 != 0])
    average_normalized_entropy[b] = - np.sum(( v0 + v1 )) / (np.log(2) * coordinates.shape[0])
    del p0, p1, v0, v1, labels, idx
    log.write("iteration_time" + " " + str(time() - t1))
    log.write("dimensionality_reduction" + " " + "None")
    log.write("average_normalized_entropy" + " " + str(float(average_normalized_entropy[b])))
    log.write("number_of_nucei" + " " + str(n))
    log.write("num_of_iterations" + " " + str(B))
  return log , classification_matrix


In [ ]:
thresholds = [0.01, 0.05, 0.1, 0.2]
ns = [50, 100, 200, 300, 1000]
for n in ns:
    for thr in thresholds:
        print("n:", n, ";", "thr:", thr)
        log , classification_matrix = teseo(n, thr, B = 500, K = 2)
        np.save(log.get_directory() + os.sep + "classification_matrix", classification_matrix)